# Prompt Engineering Notebook: Multimodal Prompting
*Google Colab–Compatible — Author: ChatGPT (o3) — Date: 2025-07-09*

This interactive notebook explores **multimodal prompting**—combining images, audio, and text within a single prompt or pipeline to unlock richer capabilities in modern foundation models.

## Learning Objectives
By the end of this notebook you will be able to:
1. Define multimodal prompting and list common modality pairs (text + image, text + audio, etc.).
2. Craft image‑conditioned prompts for vision‑language models (VLMs) such as **BLIP‑2** or GPT‑4o.
3. Implement a visual question‑answering (VQA) demo in under 20 lines of code.
4. Chain audio transcription (Whisper) with an LLM to build an audio‑aware agent.
5. Evaluate multimodal outputs for grounding, faithfulness, and bias.
6. Identify policy and safety issues unique to multimedia inputs.

## 0  | Environment Setup
Run the cell below to install lightweight dependencies. Comment out anything you already have.

In [ ]:
!pip -q install pillow transformers diffusers
# Uncomment for audio examples (heavy)
# !pip -q install git+https://github.com/openai/whisper.git --upgrade
# !pip -q install torchaudio


### Configure API Credentials (Optional)
If you have access to the **OpenAI Vision** or **Audio** endpoints, add your key below.

In [ ]:
import os, getpass
if not os.getenv('OPENAI_API_KEY'):
    os.environ['OPENAI_API_KEY'] = getpass.getpass('🔑 OpenAI API Key (optional): ')

## 1  | What Is Multimodal Prompting?
A **multimodal prompt** supplies *multiple input types*—for example, an image *and* text—to a model that natively handles those modalities or combines specialist models in a pipeline.

### Typical Patterns
| Pattern | Example | Common Models |
|---------|---------|---------------|
| **Image → Text** | Caption a photo | BLIP‑2, GPT‑4o‑Vision |
| **Image + Text → Text** | VQA: *“What color is the car in this image?”* | BLIP‑2, Gemini, LLaVA |
| **Text → Image** | *“Generate a logo of a purple owl.”* | Stable Diffusion, DALL·E 3 |
| **Audio → Text** | Transcribe a lecture | Whisper, GPT‑4o‑Audio |
| **Audio + Text → Text** | Ask follow‑up questions about a recording | Whisper + LLM chain |

## 2  | Hands‑On I – Image Captioning

> **Explain:** Image captioning converts pixels to a descriptive sentence, providing a gentle intro to vision‑language models.

In [ ]:
from PIL import Image
import requests, torch
from transformers import BlipForConditionalGeneration, BlipProcessor

device='cuda' if torch.cuda.is_available() else 'cpu'
model_name = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(model_name)
blip = BlipForConditionalGeneration.from_pretrained(model_name).to(device)

def caption_from_url(url):
    raw = Image.open(requests.get(url, stream=True).raw).convert("RGB")
    inputs = processor(raw, return_tensors="pt").to(device)
    out = blip.generate(**inputs, max_length=30)
    return processor.decode(out[0], skip_special_tokens=True)

demo_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/transformer.png"
print("Caption:", caption_from_url(demo_url))


**Exercise 🖼️**: Replace `demo_url` with any image link or upload a file via Colab sidebar, then caption it.

## 3  | Hands‑On II – Visual Question Answering (VQA)

We'll reuse BLIP‑2's Q&A head.

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
vqa_name = "Salesforce/blip2-flan-t5-xl"
processor_vqa = Blip2Processor.from_pretrained(vqa_name)
vqa_model = Blip2ForConditionalGeneration.from_pretrained(vqa_name, device_map="auto").eval()

def vqa(url, question):
    img = Image.open(requests.get(url, stream=True).raw).convert("RGB")
    inputs = processor_vqa(images=img, text=question, return_tensors="pt").to(device)
    res = vqa_model.generate(**inputs, max_length=30)
    return processor_vqa.decode(res[0], skip_special_tokens=True)

print(vqa(demo_url, "What component is highlighted?"))


**Exercise 🔍**: Ask two different questions about the same image. Compare accuracy as questions become more specific.

### Prompt Engineering Tips for VLMs
1. **Ground the question**: Reference spatial cues ("bottom left", "top center").
2. **Provide context**: *“In this technical diagram...”* improves domain grounding.
3. **Chain‑of‑thought**: Some VLMs support step‑by‑step reasoning when asked explicitly.
4. **System messages**: In GPT‑4o, prepend a system role that instructs concise answers with citations.

## 4  | Hands‑On III – Audio‑Aware Prompting

In [ ]:
#@markdown Upload a short WAV/MP3 clip via Colab and set the filename below
audio_file = "sample_audio.mp3"  # change me

try:
    import whisper
    model = whisper.load_model("base")
    result = model.transcribe(audio_file, fp16=False)
    transcript = result['text']
except Exception as e:
    transcript = "(transcript unavailable — install whisper & upload an audio file)"
print("Transcript:", transcript[:120], "...")


### Chaining with an LLM

In [ ]:
from textwrap import shorten
question = "Summarize the key points from this talk in three bullet points."
if os.getenv('OPENAI_API_KEY'):
    from langchain.llms import OpenAI
    llm = OpenAI(temperature=0)
    answer = llm(f"""Use the transcript below to answer the question.

Transcript:
"""{shorten(transcript, 400)}"""

Question: {question}
Answer:""")
else:
    answer = "(stub) Key points: 1) Example 2) Example 3) Example"
print(answer)


**Mini‑project 📑**: Build a *podcast assistant* that transcribes an episode, chunks it, and allows follow‑up Q&A.

## 5  | Evaluation & Grounding Metrics
- **Grounding**: Does the answer reference actual visual/audio evidence?
- **Faithfulness**: No invented details beyond the supplied media.
- **Relevance**: Retrieved frames/chunks aligned with question scope.
- **Bias & Fairness**: Check demographic attributes in captions.

## 6  | Safety & Policy Concerns
- **Sensitive imagery**: NSFW, violent, or private images require filters.
- **Faces & PII**: Avoid identifying real individuals without consent.
- **Audio privacy**: Recordings may contain personal data.
- **Copyright**: Ensure you have rights to distribute images/audio used in demos.

## Assignment 🎓
Design a multimodal tutor that helps students learn geography by:
1. Accepting an image of a world map section.
2. Answering *three* progressively harder questions about the region.
3. Providing follow‑up resources with image URLs.
4. Logging each interaction with timestamp and question difficulty.

## Further Reading & Resources
- Li et al., *BLIP‑2: Bootstrapping Language‑Image Pre‑training* (2023)
- OpenAI *Vision* & *Audio* docs (GPT‑4o)
- *LLaVA*: Large‑Language‑and‑Vision Assistant (2023)
- Diffusers library (Hugging Face) for text‑to‑image prompting
- Microsoft *Kosmos‑2* multimodal grounding